In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import nltk
import re

## Datenvorbereitung

In [43]:
df = pd.read_excel("data_analyst_recommendations.xlsx")

In [44]:
def convert_date_format(date_str):
    if date_str == "Fr":
        # Ersetzen Sie "Fr" durch ein leeres Zeichen
        date_str = date_str.replace("Fr", "02.02.").strip()

    # Überprüfen Sie, ob das Jahr bereits vorhanden ist
    if len(date_str.split(".")[2]) == 0:
        # Fügen Sie das Jahr hinzu (nehmen Sie das aktuelle Jahr an, wenn nicht angegeben)
        date_str = f"{date_str}{datetime.now().year}"
        date_str = datetime.strptime(date_str, "%d.%m.%Y")
    else:
        date_str = datetime.strptime(date_str, "%d.%m.%y")

    # Formatieren Sie das Datum für die Ausgabe
    return date_str.strftime("%d.%m.%Y")

In [45]:
formatted_dates = [convert_date_format(date) for date in df["date"]]
df["date"] = formatted_dates

In [46]:
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y")

In [47]:
df.set_index("date", inplace=True)

## Datenbereinigung

In [48]:
cities = [city.split(" - ")[0] for city in df["recommendation"]]
recommendations = [recommendation.split(" - ")[1] for recommendation in df["recommendation"]]

In [49]:
recommendations_cleaned = [recommendation.split("./")[0] for recommendation in recommendations]

In [50]:
cities_cleaned = [city.split(" (")[0] for city in cities]

In [51]:
df["recommendation"] = recommendations_cleaned

In [52]:
df["city"] = cities_cleaned

## Stoppwörter ausschließen

In [73]:
# Extrahieren aller label aus den Titeln, um diese als Stopwörter auszuschließen
stopwords = set()

for title in df["title"]:
    stopwords.update(re.findall(r"'([^']*)'", title))

In [76]:
# Definition einer Liste mit diesen Stoppwörtern
custom_stopwords = [word.lower() for word in stopwords]

In [82]:
# Manuelle Ergänzungen der Liste
custom_stopwords.append("''")
custom_stopwords.append("``")
custom_stopwords

['outperform',
 'neutral',
 'hold',
 'overweight',
 'underperform',
 'buy',
 'kaufen',
 'underweight',
 "''",
 "''",
 '``']

In [83]:
filtered_text_list = []

In [84]:
# Tokenisieren Sie den Text
for text in df["recommendation"]:
    # Tokenisieren der Empfehlungen
    tokenized_text = nltk.word_tokenize(text)

    # Stopwörter aus Text entfernen
    filtered_tokens = [word for word in tokenized_text if word.lower() not in custom_stopwords]

    # Verbinden der verbleibenden Wörter zu einem Text
    filtered_text = ' '.join(filtered_tokens)

    filtered_text_list.append(filtered_text)

In [87]:
filtered_text_list_cleaned = [text.replace(" .", ".").replace(" ,", ",") for text in filtered_text_list]

In [89]:
df["recommendation_cleaned"] = filtered_text_list_cleaned

In [90]:
df

,title,link,recommendation,city,recommendation_cleaned
date,,,,,
2024-02-02,DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,Deutsche Bank Research hat die Einstufung für ...,FRANKFURT,Deutsche Bank Research hat die Einstufung für ...
2024-01-23,BERNSTEIN RESEARCH stuft SIEMENS AG auf 'Outpe...,https://www.finanznachrichten.de/nachrichten-2...,Das US-Analysehaus Bernstein Research hat die ...,NEW YORK,Das US-Analysehaus Bernstein Research hat die ...
2024-01-22,JPMORGAN stuft SIEMENS AG auf 'Overweight',https://www.finanznachrichten.de/nachrichten-2...,Die US-Bank JPMorgan hat die Einstufung für Si...,NEW YORK,Die US-Bank JPMorgan hat die Einstufung für Si...
2024-01-22,DZ BANK stuft SIEMENS AG auf 'Kaufen',https://www.finanznachrichten.de/nachrichten-2...,Die DZ Bank hat den fairen Wert für die Aktien...,FRANKFURT,Die DZ Bank hat den fairen Wert für die Aktien...
2024-01-15,JPMORGAN stuft SIEMENS AG auf 'Overweight',https://www.finanznachrichten.de/nachrichten-2...,Die US-Bank JPMorgan hat die Einstufung für Si...,NEW YORK,Die US-Bank JPMorgan hat die Einstufung für Si...
...,...,...,...,...,...
2023-02-09,DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,Deutsche Bank Research hat das Kursziel für Si...,FRANKFURT,Deutsche Bank Research hat das Kursziel für Si...
2023-02-09,BARCLAYS stuft SIEMENS AG auf 'Underweight',https://www.finanznachrichten.de/nachrichten-2...,Die britische Investmentbank Barclays hat die ...,LONDON,Die britische Investmentbank Barclays hat die ...
2023-02-09,GOLDMAN SACHS stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,Die US-Investmentbank Goldman Sachs hat Siemen...,NEW YORK,Die US-Investmentbank Goldman Sachs hat Siemen...


## FinBERT

In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model="scherrmann/GermanFinBert_SC_Sentiment")

## Sonstiges

In [42]:
pipe(filtered_text)

[{'label': 'Negativ', 'score': 0.9872450232505798}]

In [41]:
# Beispieltext
text = 'Die US-Investmentbank Morgan Stanley hat die Einstufung für Siemens auf "Overweight" mit einem Kursziel von 164 Euro belassen. Die nachlassende Nachfrage im europäischen Investitionsgütersektor stelle ein Problem für das zweite Halbjahr dar, schrieb Analyst Ben Uglow in einer am Mittwoch vorliegenden Branchenstudie. Die normalisierten Lieferketten sorgten für den Abbau von Lagerbeständen bei den Kunden. Zudem bestünden Risiken für die operativen Margen im kommenden Jahr'

# Regulärer Ausdruck für Wörter, die auf "*wachstum" enden
pattern = re.compile(r'\b(?:gewinn|umsatz|\w*wachstum)\b', flags=re.IGNORECASE)

# Finden Sie alle Übereinstimmungen
matches = pattern.findall(text.lower())

# Ihre eigene Liste von Stopwörtern
custom_stopwords = ['``', "''", "underweight", "wachstum", "buy", "overweight", "outperform", "underperform"]

# Fügen Sie die gefundenen Übereinstimmungen zu den Stopwörtern hinzu
custom_stopwords += matches

# Tokenisieren Sie den Text
tokens = nltk.word_tokenize(text)

# Entfernen Sie die Stopwörter
filtered_tokens = [word for word in tokens if word.lower() not in custom_stopwords]

# Verbinden Sie die verbleibenden Wörter zu einem Text
filtered_text = ' '.join(filtered_tokens)

print(filtered_text)
print(matches)

Die US-Investmentbank Morgan Stanley hat die Einstufung für Siemens auf mit einem Kursziel von 164 Euro belassen . Die nachlassende Nachfrage im europäischen Investitionsgütersektor stelle ein Problem für das zweite Halbjahr dar , schrieb Analyst Ben Uglow in einer am Mittwoch vorliegenden Branchenstudie . Die normalisierten Lieferketten sorgten für den Abbau von Lagerbeständen bei den Kunden . Zudem bestünden Risiken für die operativen Margen im kommenden Jahr
[]
